Numerical methods can converge to entropy-violating solutions.  Here we consider Example 12.3 from [LeVeque 1992].  We take Burgers' equation

$$q_t + \left(\frac{1}{2} q^2\right)_x = 0$$

with Riemann initial data

$$q(x,0) = \begin{cases} -1 & x < 0 \\\\ 1 & x > 0 \end{cases}.$$

We'll discretize using flux-differencing

$$Q^{n+1}_j = Q^n_j - \frac{\Delta t}{\Delta x} \left(F_{i+1/2} - F_{i-1/2}\right)$$

where the numerical flux is taken as the upwind flux:

$$F_{i-1/2} = \begin{cases} f(Q_{i-1}) & \frac{Q_{i}+Q_{i-1}}{2}\ge0 \\\\
f(Q_{i}) & \frac{Q_{i}+Q_{i-1}}{2}<0\end{cases}$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time, sys
import matplotlib
from matplotlib import animation
from JSAnimation import IPython_display

In [ ]:
def take_step(q,dt,dx,entropy_fix=False):
    f = np.zeros(q.shape) # fluxes
    avg = (q[:-2] + q[1:-1])/2.
    f[1:-1] = 0.5*q[:-2]**2 * (avg>0.) + 0.5*q[1:-1]**2 * (avg<=0.)  # F_{i-1/2}
    
    if entropy_fix:
        for i in range(len(f)):
            if q[i]>0. and q[i-1]<0.:
                f[i] = 0.
    
    q[1:-1] = q[1:-1] - dt/dx * ( f[2:] - f[1:-1] )
    return q

In [ ]:
cflnum=0.45;
N=1000;  # Number of grid points
plot_interval = N/10  # Draw a plot every n time steps

dx=1./N;
nghost = 2; N2=N+2*nghost;  # Ghost cells used for implementing boundary conditions
x=np.linspace(-(nghost-0.5)*dx,1.+(nghost-0.5)*dx,N2)
t=0.; T=0.5; # Initial and final time

q0 = -1.*(x<0.5) + 1.*(x>0.5)
#q = q0.copy();

dt=cflnum*dx/np.max(q0);

# Set up plotting
fig = plt.figure()
axes = fig.add_subplot(111)
line1, = axes.plot(x,q0,lw=2)
line2, = axes.plot(x,q0,'r--',lw=2)
axes.set_xlim(0.,1.)
axes.legend(['Without entropy fix','With entropy fix'],loc='upper left')

frames1 = [q0.copy()]
frames2 = [q0.copy()]
q_n = q0.copy()
q_e = q0.copy()

i = 0
while t<T:
    if dt>T-t: dt=T-t
        
    q_n = take_step(q_n ,dt, dx, entropy_fix=False)
    q_e = take_step(q_e ,dt, dx, entropy_fix=True)

    for q in [q_n, q_e]:  # Fixed boundary conditions
        q[0:nghost]  = -1. 
        q[-nghost:]  = 1.
        
    t = t + dt
    i = i + 1
    if (i % plot_interval) == 0:
        frames1.append(q_n.copy())
        frames2.append(q_e.copy())
        
def plot_frame(i):
    line1.set_data(x,frames1[i])
    line2.set_data(x,frames2[i])

matplotlib.animation.FuncAnimation(fig, plot_frame, frames=len(frames1), interval=200)

What happens if we use an odd number of grid points?

In [ ]:
axes.legend?